In [ ]:
# Takes the output from routing_3.ipynb and works on the routing portion of it
import polars as pl
import numpy as np
import os
import rtsvg
rt = rtsvg.RACETrack()

In [ ]:
df_collapsed    = pl.read_parquet('../../data/stanford/facebook/348.edges_collapsed_edges.3.parquet')
df_edge_arc_pos = pl.read_parquet('../../data/stanford/facebook/348.edges_arc_pos.3.parquet')
df_collapsed.sample(1)

In [ ]:
df_edge_arc_pos.sample(1)

In [ ]:
def extractCircles(_df_):
    circles, circle_lu = [], {}
    for k, k_df in _df_.group_by(['x_circle','y_circle','r_circle']):
        circle_lu[k] = len(circles)
        circles.append(k)
    return circles, circle_lu
def validatePositions(_df_, clearance=10): # edge_arc_pos
    # Extract the circles first
    circles, circle_lu = extractCircles(_df_)
    # Then make sure the positions don't fall within a circle & that they have some clearance
    for k, k_df in _df_.group_by(['x','y', 'x_circle', 'y_circle', 'r_circle']):
        _xy_     = (k[0], k[1])
        _uv_     = rt.unitVector((k[2:4], _xy_))
        _xy_end_ = (_xy_[0]+ clearance*_uv_[0], _xy_[1]+ clearance*_uv_[1])
        _my_circle_ = k[2:5]
        for c in circles:
            if c == _my_circle_: continue
            _length_ = rt.segmentLength((_xy_,c))
            if _length_ < c[2] + clearance:
                raise Exception(f'node key "{k_df["node_key"]}" with radius circle {c}')
            _length_ = rt.segmentLength((_xy_end_, c))

def extents(_df_): # edge_arc_pos -- assumes that the coordinates are already in screen space
    xmax, ymax = 10.0, 10.0
    for k, k_df in _df_.group_by(['x_circle','y_circle','r_circle']):
        xmax, ymax = max(xmax, k[0]+k[2]), max(ymax, k[1]+k[2])
    for k, k_df in _df_.group_by(['x','y']):
        xmax, ymax = max(xmax, k[0]), max(ymax, k[1])
    return (0.0, 0.0, xmax, ymax)

validatePositions(df_edge_arc_pos)
_ext_ = extents(df_edge_arc_pos)

In [ ]:
x_ins, y_ins = 50, 50
# Base Circles
svg_base = [f'<svg x="0" y="0" width="{_ext_[2]+x_ins}" height="{_ext_[3]+y_ins}">']
svg_base.append(f'<rect x="0" y="0" width="{_ext_[2]+x_ins}" height="{_ext_[3]+y_ins}" fill="#ffffff" />')
circles, circles_lu = extractCircles(df_edge_arc_pos)
for i in range(len(circles)):
    cx, cy, r = circles[i]
    svg_base.append(f'<circle cx="{cx}" cy="{cy}" r="{r}" fill="none" stroke="{rt.co_mgr.getColor(i)}" stroke-width="3" />')
# Entry / Exit Points
svg_pts = []
for k, k_df in df_edge_arc_pos.group_by(['node_key', 'x', 'y', 'x_circle', 'y_circle', 'r_circle']):
    _xy_, _cxy_ = k[1:3], k[3:5]
    svg_pts.append(f'<circle cx="{_xy_[0]}" cy="{_xy_[1]}" r="4" fill="none" stroke="#505050" stroke-width="0.5"/>')
    _uv_        = rt.unitVector((_cxy_, _xy_))
    svg_pts.append(f'<line x1="{_xy_[0]}" y1="{_xy_[1]}" x2="{_xy_[0]+10*_uv_[0]}" y2="{_xy_[1]+10*_uv_[1]}" stroke="#404040" stroke-width="1.0"/>')
# Delauney Triangulations
_box_ = [(_ext_[0],_ext_[1]),(_ext_[0],_ext_[3]+y_ins),(_ext_[2]+x_ins,_ext_[3]+y_ins),(_ext_[2]+x_ins,_ext_[1])]
voronoi_polys = rt.isedgarVoronoi(circles,Box=_box_)
svg_voronoi = []
for i in range(len(voronoi_polys)):
    d = f'M {voronoi_polys[i][0][0]} {voronoi_polys[i][0][1]} '
    for j in range(1, len(voronoi_polys[i])): d += f'L {voronoi_polys[i][j][0]} {voronoi_polys[i][j][1]} '
    d += 'Z'
    svg_voronoi.append(f'<path d="{d}" fill="none" stroke="{rt.co_mgr.getColor(i)}" stroke-width="3"/>')

rt.tile([''.join(svg_base)+''.join(svg_pts)+''.join(svg_voronoi)+'</svg>'])